In [9]:
from bs4 import BeautifulSoup as bs
import requests
import json
import csv
from pprint import pprint
import pandas as pd

# https://podolsk.hh.ru/search/vacancy?clusters=true&no_magic=true&
# ored_clusters=true&enable_snippets=true&salary=&st=searchVacancy&
# text=Python&from=suggest_post


def hh_vacancies():
    my_vacancy = input('Input your desired vacancy: ')
    page_num = 0
    while page_num != '':
        url = 'https://hh.ru'
        params = {
            'clusters': 'true',
            'no_magic': 'true',
            'ored_clusters': 'true',
            'enable_snippets': 'true',
            'st': 'searchVacancy',
            'text': f'{my_vacancy}',
            'from': 'suggest_post',
            'page': f'{page_num}'
        }
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                                  AppleWebKit/537.36 (KHTML, like Gecko)\
                                  Chrome/90.0.4430.93 Safari/537.36'}

        response = requests.get(url+'/search/vacancy', params=params, headers=headers)
        if response.ok:
            soup = bs(response.text, 'html.parser')
            vacancies_list = soup.find_all('div', {'class': 'vacancy-serp-item'})

            def salary_dev(x):
                salary_data = {}  # Словарь для хранения полученных данных
                if x != '':
                    val = x.split()[-1]  # Извлечение валюты
                    num_min = None
                    num_max = None

                    if '–' in x:  # Извлечение мин и макс зарплаты, если есть оба значения
                        num = x.split('–')
                        num_min = int((''.join(i for i in num[0] if i.isdigit())))
                        num_max = int((''.join(i for i in num[1] if i.isdigit())))
                    elif 'от' in x:  # Извлечение мин зарплаты, если есть одно значение
                        num_min = int((''.join(i for i in x if i.isdigit())))
                    elif 'до' in x:  # Извлечение макс зарплаты, если есть одно значение
                        num_max = int((''.join(i for i in x if i.isdigit())))
            # Заполнение словаря
                    salary_data['min'] = num_min
                    salary_data['max'] = num_max
                    salary_data['carency'] = val
                else:
                    salary_data['min'] = None
                    salary_data['max'] = None
                    salary_data['carency'] = None
                return salary_data

            vacancies = []
            for vacancy in vacancies_list:
                vacancy_data = {}
                vacancy_info = vacancy.find('a', {'class': 'bloko-link'})
                vacancy_name = vacancy_info.getText()
                vacancy_url = vacancy_info.get('href')
                vacancy_salary = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
                vacancy_salary_dev = salary_dev(vacancy_salary)
                site_url = url

                vacancy_data['name'] = vacancy_name
                vacancy_data['url'] = vacancy_url
                vacancy_data['salary'] = vacancy_salary_dev
                vacancy_data['site_url'] = site_url

                vacancies.append(vacancy_data)

            with open('hh_vacancies.csv', 'a', encoding='UTF-8', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=vacancies[0])
                writer.writeheader()
                writer.writerows(vacancies)
            
            page_num += 1
            
        else:
            break
hh_vacancies()

Input your desired vacancy: python


In [10]:

#print(b)
path = 'hh_vacancies.csv'
df = pd.read_csv(path)
df.head()

,name,url,salary,site_url
0,Middle Python/Django Developer (в г. Новосибирск),https://hh.ru/vacancy/43531154?from=vacancy_se...,"{'min': None, 'max': None, 'carency': None}",https://hh.ru
1,Senior Python Developer,https://hh.ru/vacancy/46423378?from=vacancy_se...,"{'min': None, 'max': None, 'carency': None}",https://hh.ru
2,"Full Stack Developer (Python/R, JS/TS)",https://hh.ru/vacancy/46274567?from=vacancy_se...,"{'min': None, 'max': None, 'carency': None}",https://hh.ru
3,Fullstack Python Developer,https://hh.ru/vacancy/46242541?from=vacancy_se...,"{'min': None, 'max': None, 'carency': None}",https://hh.ru
4,Data Analyst/ Data Scientist,https://hh.ru/vacancy/45693700?from=vacancy_se...,"{'min': None, 'max': None, 'carency': None}",https://hh.ru


In [12]:
df.shape

(860, 4)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      860 non-null    object
 1   url       860 non-null    object
 2   salary    860 non-null    object
 3   site_url  860 non-null    object
dtypes: object(4)
memory usage: 27.0+ KB
